In [1]:
###########################################
# IMPORTS
###########################################
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactoryV3
from perses.annihilation.lambda_protocol import RESTStateV3
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

#############################################
# CONSTANTS
#############################################
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("Reference")

INFO:rdkit:Enabling RDKit 2021.03.5 jupyter extensions


conducting subsequent work with the following platform: CPU


In [2]:
def compare_energy_components(rest_system, other_system, positions, platform=REFERENCE_PLATFORM):
    """
    Get energy components of a given system
    """
    platform = configure_platform(platform)

    # Create thermodynamic state and sampler state for non-rest system
    thermostate_other = ThermodynamicState(system=other_system, temperature=temperature)

    # Create context for non-rest system
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)

    # Get energy components for non-rest system
    components_other = [component for component in compute_potential_components(context_other, beta=beta)]
    
    # Create thermodynamic state for rest_system
    thermostate_rest = ThermodynamicState(system=rest_system, temperature=temperature)

    # Create context for rest system
    integrator_rest = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_rest = thermostate_rest.create_context(integrator_rest)
    context_rest.setPositions(positions)

    # Get energy components for rest system
    components_rest = [component for component in compute_potential_components(context_rest, beta=beta)]

    print(components_other)
    print(components_rest)
        
    # Check that bond, angle, and torsion energies match
    for other, rest in zip(components_other[:3], components_rest[:3]):
        assert np.isclose([other[1]], [rest[1]]), f"The energies do not match for the {other[0]}: {other[1]} (other system) vs. {rest[1]} (REST system)"

    # Check that nonbonded energies match
    nonbonded_other = np.array([component[1] for component in components_other[3:]]).sum()
    nonbonded_rest = np.array([component[1] for component in components_rest[3:]]).sum()
    print(nonbonded_other)
    print(nonbonded_rest)
    assert np.isclose([nonbonded_other], [nonbonded_rest]), f"The energies do not match for the NonbondedForce: {nonbonded_other} (other system) vs. {nonbonded_rest} (REST system)"
    
    print("Energy bookkeeping was a success!")
    
    


## Alanine dipeptide in vacuum

In [27]:
# Create vanilla system
ala = AlanineDipeptideVacuum()
system = ala.system
positions = ala.positions
topology = ala.topology



In [28]:
# Create REST system
system.removeForce(4)
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 0
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]


In [29]:
# Compare energy components
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03455064996280121), ('HarmonicAngleForce', 0.6071369391574322), ('PeriodicTorsionForce', 3.2298482876455963), ('NonbondedForce', -39.18693169158183), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455064996280121), ('CustomAngleForce', 0.6071369391574319), ('CustomTorsionForce', 3.2298482876455954), ('CustomNonbondedForce', -134.40418350398505), ('CustomNonbondedForce', 4.716823991270119), ('CustomBondForce', 90.50042198031858), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
-39.18693169158183
-39.18693753239634
Energy bookkeeping was a success!


## Alanine dipeptide in solvent

In [30]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology



In [31]:
# Create REST system
res1 = list(ala.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


In [5]:
REST_system.getForces()

[<openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x10eb59720> >,
 <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x190951e70> >,
 <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x19083ccc0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x10ebb63f0> >,
 <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x10ebb6f30> >,
 <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x10ebb68d0> >,
 <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x10ebb6060> >]

In [6]:
rest_atoms

[6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [37]:
# system.getForce(3).setIncludeDirectSpace(False)

In [32]:
system.getForce(3).setReciprocalSpaceForceGroup(31) # Only show direct space energy

In [33]:
for i in range(system.getForce(3).getNumParticles()):
    charge, sigma, epsilon = system.getForce(3).getParticleParameters(i)
#     if i > 21:
#         system.getForce(3).setParticleParameters(i, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setParticleParameters(i, charge, sigma, epsilon*0)


In [34]:
for i in range(system.getForce(3).getNumExceptions()):
    p1, p2, charge, sigma, epsilon = system.getForce(3).getExceptionParameters(i)
#     if p1 > 21 or p2 > 21:
#         system.getForce(3).setExceptionParameters(i, p1, p2, charge*0, sigma, epsilon*0)
#     else:
    system.getForce(3).setExceptionParameters(i, p1, p2, charge, sigma, epsilon*0)
    

In [8]:
# for i in range(REST_system.getForce(3).getNumParticles()):
#     params = REST_system.getForce(3).getParticleParameters(i)
# #     if i > 21:
#     params = list(params)
#     params[-3] = params[-3]*0
#     params[-1] = params[-1]*0
#     REST_system.getForce(3).setParticleParameters(i, params)


In [35]:
for i in range(REST_system.getForce(5).getNumBonds()):
    p1, p2, params = REST_system.getForce(5).getBondParameters(i)
#     if p1 > 21 or p2 > 21:
#         params = list(params)
#     #     params[-3] = params[-3]*0
#         params[-2] = params[-2]*0
#         REST_system.getForce(5).setBondParameters(i, p1, p2, params)
#     else:
    params = list(params)
#     params[-4] = params[-4]*0
    params[-2] = params[-2]*0
    REST_system.getForce(5).setBondParameters(i, p1, p2, params)


In [36]:
# Compare energy components
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03454256736202116), ('HarmonicAngleForce', 0.6071512895607732), ('PeriodicTorsionForce', 3.2298499815095068), ('NonbondedForce', 69477.3519953855), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03454256736202116), ('CustomAngleForce', 0.6071512895607735), ('CustomTorsionForce', 3.2298499815095063), ('CustomNonbondedForce', -10097.515279207637), ('CustomNonbondedForce', 1203.1377794099421), ('CustomBondForce', 79574.86995277482), ('NonbondedForce', -80578.88657404146), ('AndersenThermostat', 0.0)]
69477.3519953855
-9898.394121064324


AssertionError: The energies do not match for the NonbondedForce: 69477.3519953855 (other system) vs. -9898.394121064324 (REST system)

In [35]:
for k, v in context.getParameters().items():
    print(k, v)

AndersenCollisionFrequency 1.0
AndersenTemperature 300.0
lambda_rest_angles 1.0
lambda_rest_bonds 1.0
lambda_rest_electrostatics 1.0
lambda_rest_electrostatics_exceptions 1.0
lambda_rest_sterics 1.0
lambda_rest_sterics_exceptions 1.0
lambda_rest_torsions 1.0


# Create two particle system

In [5]:
from simtk.openmm import app

In [6]:
system = openmm.System()
system.addParticle(22.99)
system.addParticle(35.45)
system.setDefaultPeriodicBoxVectors(openmm.Vec3(9, 0, 0), openmm.Vec3(0, 9, 0), openmm.Vec3(0, 0, 9))

system.addForce(openmm.HarmonicBondForce())
system.addForce(openmm.HarmonicAngleForce())
system.addForce(openmm.PeriodicTorsionForce())

nb_force = openmm.NonbondedForce()
system.addForce(nb_force)
nb_force.setNonbondedMethod(openmm.NonbondedForce.PME)
nb_force.setCutoffDistance(0.3 * unit.nanometer)
nb_force.addParticle(1.0, 0.24, 0.37)
nb_force.addParticle(-1.0, 0.45, 0.15)
positions = [openmm.Vec3(0.1, 0, 0), openmm.Vec3(0.3, 0, 0)]

In [7]:
# Create new topology
topology = app.Topology()

# Copy residues and atoms to new topology for chains_to_keep
chain = topology.addChain(id='A')
res = topology.addResidue('NA', chain, id='1')
atom = topology.addAtom('Na', openmm.app.element.sodium, res)
res = topology.addResidue('CL', chain, id='1')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)

In [8]:
# Create REST system
factory = RESTTopologyFactoryV3(system, topology, rest_region=[0], use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 8.760869616261553
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=9.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=9.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=9.0), unit=nanometer)]


In [9]:
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.0), ('HarmonicAngleForce', 0.0), ('PeriodicTorsionForce', 0.0), ('NonbondedForce', -26.206667676031483), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.0), ('CustomAngleForce', 0.0), ('CustomTorsionForce', 0.0), ('CustomNonbondedForce', -3.680127136351365), ('CustomNonbondedForce', 252.29961791797672), ('CustomBondForce', 0.0), ('NonbondedForce', -274.8260823793764), ('AndersenThermostat', 0.0)]
-26.206667676031483
-26.206591597751014
Energy bookkeeping was a success!


# Create three particle system

In [3]:
from simtk.openmm import app

In [4]:
system = openmm.System()
system.addParticle(22.99)
system.addParticle(35.45)
system.addParticle(35.45)
system.setDefaultPeriodicBoxVectors(openmm.Vec3(1, 0, 0), openmm.Vec3(0, 1, 0), openmm.Vec3(0, 0, 1))

system.addForce(openmm.HarmonicBondForce())
system.addForce(openmm.HarmonicAngleForce())
system.addForce(openmm.PeriodicTorsionForce())

nb_force = openmm.NonbondedForce()
system.addForce(nb_force)
nb_force.setNonbondedMethod(openmm.NonbondedForce.PME)
nb_force.setCutoffDistance(0.3 * unit.nanometer)
nb_force.addParticle(1.0, 0.24, 0.37)
nb_force.addParticle(-0.5, 0.45, 0.15)
nb_force.addParticle(-0.5, 0.45, 0.15)
nb_force.addException(1, 2, 0.5, (0.45+0.45)/2, np.sqrt(0.15*0.15))

positions = [openmm.Vec3(0.3, 0, 0), openmm.Vec3(0.1, 0, 0), openmm.Vec3(0.5, 0, 0)]

In [5]:
# Create new topology
topology = app.Topology()

# Copy residues and atoms to new topology for chains_to_keep
chain = topology.addChain(id='A')
res = topology.addResidue('NA', chain, id='1')
atom = topology.addAtom('Na', openmm.app.element.sodium, res)
res = topology.addResidue('CL', chain, id='2')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)
res = topology.addResidue('CL', chain, id='3')
atom = topology.addAtom('Cl', openmm.app.element.chlorine, res)

In [6]:
# Create REST system
factory = RESTTopologyFactoryV3(system, topology, rest_region=[0], use_dispersion_correction=True)
REST_system = factory.REST_system



INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 8.760869616261553
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=1.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=1.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=1.0), unit=nanometer)]


In [7]:
context = compare_energy_components(REST_system, system, positions)


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.0), ('HarmonicAngleForce', 0.0), ('PeriodicTorsionForce', 0.0), ('NonbondedForce', 299.02125594493225), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.0), ('CustomAngleForce', 0.0), ('CustomTorsionForce', 0.0), ('CustomNonbondedForce', -3.680129314760381), ('CustomNonbondedForce', 506.5022689500185), ('CustomBondForce', 35.31368190765442), ('NonbondedForce', -239.11428519209545), ('AndersenThermostat', 0.0)]
299.02125594493225
299.02153635081703
Energy bookkeeping was a success!


# Run all tests

In [7]:
def test_bookkeeping():
    """
    Given the default Global variables, do energy component bookkeeping with the REST system and the original system.
    Specifically, the first CustomBondForce must match the HarmonicBondForce
    (same with the second and third forces with the HarmonicAngle and PeriodicTorsionForces).
    The sum of the NonbondedForce, the CustomNonbondedForce, and the CustomBondForce (last one) must be equal to the
    energy of the original system's NonbondedForce.
    """

    ## CASE 1: alanine dipeptide in vacuum
    # Create vanilla system
    ala = AlanineDipeptideVacuum()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 2: alanine dipeptide in solvent
    # Create vanilla system
    ala = AlanineDipeptideExplicit()
    system = ala.system
    positions = ala.positions
    topology = ala.topology

    # Create REST system
    system.removeForce(4)
    res1 = list(ala.topology.residues())[1]
    rest_atoms = [atom.index for atom in res1.atoms()]
    factory = RESTTopologyFactoryV3(system, topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, system, positions)

    ## CASE 3: alanine dipeptide in solvent with repartitioned hybrid system
    # Create repartitioned hybrid system for lambda 0 endstate
    atp, system_generator = generate_atp(phase='solvent')
    htf = generate_dipeptide_top_pos_sys(atp.topology,
                                         new_res='THR',
                                         system=atp.system,
                                         positions=atp.positions,
                                         system_generator=system_generator,
                                         conduct_htf_prop=True,
                                         repartitioned=True,
                                         endstate=0,
                                         validate_endstate_energy=False)
    
    # Create REST-ified hybrid system
    res1 = list(htf.hybrid_topology.residues)[1]
    rest_atoms = [atom.index for atom in list(res1.atoms)]
    factory = RESTTopologyFactoryV3(htf.hybrid_system, htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
    REST_system = factory.REST_system

    # Compare energy components
    compare_energy_components(REST_system, htf.hybrid_system, htf.hybrid_positions)


In [8]:
test_bookkeeping()

INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 0
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 3.2893431387452243
INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03455064996280121), ('HarmonicAngleForce', 0.6071369391574322), ('PeriodicTorsionForce', 3.2298482876455963), ('NonbondedForce', -39.18693169158183), ('AndersenThermostat', 0.0)]
[('CustomBondForce', 0.03455064996280121), ('CustomAngleForce', 0.6071369391574319), ('CustomTorsionForce', 3.2298482876455954), ('CustomNonbondedForce', -134.40418350398502), ('CustomNonbondedForce', 4.716823991270118), ('CustomBondForce', 90.50042198031858), ('NonbondedForce', 0.0), ('AndersenThermostat', 0.0)]
-39.18693169158183
-39.18693753239634
Energy bookkeeping was a success!
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 0.03454256736202116), ('Ha

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]


generating htf
making topology proposal


INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [10, 14, 18, 13, 17, 16, 19, 15]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in reference force.
INFO:geometry:	creating angle force...
INFO:geometry:	there are 43 angles in reference force.
INFO:geometry:	creating torsion force...
INFO:geometry:	creating extra torsions force...
INFO:geometry:	there are 72 torsions in reference force.
INFO:geometry:	creating nonbonded force...
INFO:geometry:		grabbing reference nonbonded method, cutoff, switching fun

...

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 1.0558269152414181), ('CustomAngleForce', 15.749061874823555), ('CustomTorsionForce', 19.193719397769534), ('CustomBondForce', -59.95290763572909)]
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	reduced angle potential = 3.2058324695160406e-13.
INFO:geometry:	reduced angle potential = 1.2915588460963948e-10.
INFO:geometry:	reduced angle potential = 7.39096069988752e-11.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 9 bond forces in the no-nonbonded final system
INFO:geometry:	there are 36 angle forces in the no-nonbonded final system
INFO:geometry:	there are 42 torsion forces in the no-nonbonded final system
INFO:geometry:reverse final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 9.116481158706167
INFO:geometry:total reduced energy added from growth 

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.0005201491202417058), ('CustomAngleForce', 0.4511199297896573), ('CustomTorsionForce', 7.25046198458677), ('CustomBondForce', 12.970881401086856)]


INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.918423065872431
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.56477354, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.56477354, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.56477354), unit=nanometer)]


generating rest system
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 1.0903955094574882), ('HarmonicAngleForce', 16.356284407942475), ('PeriodicTorsionForce', 35.37057219695222), ('NonbondedForce', -998.326288790699), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
[('CustomBondForce', 1.0903955094574882), ('CustomAngleForce', 16.35628440794247), ('CustomTorsionForce', 35.37057219695221), ('CustomNonbondedForce', -7418.759680689047), ('CustomNonbondedForce', 7032.4635409539505), ('CustomBondForce', 48354.465585379934), ('NonbondedForce', -48966.49698156664), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
-998.326288790699
-998.3275359218023
Energy bookkeeping was a success!


# Test barnase:barstar

In [14]:
from perses.app.relative_point_mutation_setup import PointMutationExecutor
from openeye import oechem

In [24]:
solvent_delivery = PointMutationExecutor("../../input/mmc2_barstar.pdb",
                        '1',
                        "42",
                        "ALA",
                        ligand_input="../../input/mmc2_barnase.pdb",
                        ionic_strength=0.05*unit.molar,
                        flatten_torsions=True,
                        flatten_exceptions=True, 
                        conduct_endstate_validation=False,
                        generate_rest_capable_hybrid_topology_factory=True
                       )

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11
INFO:setup:solvating at 0.05 M using tip3p
INFO:setup:solvating at 0.05 M using tip3p
INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=CG2, atomic number=6), Atom(name=OG1, atomic number=8), Atom(name=HB, atomic number=1), Atom(name=HG1, atomic number=1), Atom(name=HG21, atomic number=1), Atom(name=HG22, atomic number=1), Atom(name=HG23, atomic number=1)]
INFO:proposal_generator:[Atom(name=CB, atomic number=6), Atom(name=HB1, atomic number=1), Atom(name=HB2, atomic number=1), Atom(name=HB3, atomic number=1)]
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 4
INFO:geometry:Atom index prop

conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	reduced angle potential = 0.606500164772652.
INFO:geometry:	reduced angle potential = 6.332550869284324.
INFO:geometry:	reduced angle potential = 0.0037414881038030957.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 733 bond forces in the no-nonbonded final system
INFO:geometry:	there are 2628 angle forces in the no-nonbonded final system
INFO:geometry:	there are 4939 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 2279.2365641521073


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced energy added from growth system: 575.6812110066791
INFO:geometry:final reduced energy 2854.917775252372
INFO:geometry:sum of energies: 2854.9177751587863
INFO:geometry:magnitude of difference in the energies: 9.358552688354393e-08
INFO:geometry:Final logp_proposal: 27.77556824512547
INFO:geometry:logp_reverse: performing reverse proposal
INFO:geometry:logp_reverse: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 8
INFO:geometry:Atom index proposal order is [673, 675, 674, 678, 681, 679, 682, 680]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is reverse; creating atoms_with_positions from old system/topology


...

INFO:geometry:	reduced angle potential = 0.1536280916466212.
INFO:geometry:	reduced angle potential = 0.22571252374478284.
INFO:geometry:	reduced angle potential = 0.00669552145106419.
INFO:geometry:	reduced angle potential = 0.023352265895512498.
INFO:geometry:	reduced angle potential = 0.05657264681042365.
INFO:geometry:	reduced angle potential = 0.00589659951395519.
INFO:geometry:	reduced angle potential = 0.25876654048332753.
INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 1624 bond forces in the no-nonbonded final system
INFO:geometry:	there are 5737 angle forces in the no-nonbonded final system
INFO:geometry:	there are 10822

conducting subsequent work with the following platform: CPU


INFO:geometry:total reduced potential before atom placement: 4792.466756866703
INFO:geometry:total reduced energy added from growth system: -62.390092842368496
INFO:geometry:final reduced energy 4730.076663024306
INFO:geometry:sum of energies: 4730.076664024335
INFO:geometry:magnitude of difference in the energies: 1.0000282131272797e-06
INFO:geometry:Final logp_proposal: -9175.545028786522
INFO:setup:charge diff: 0
INFO:relative:*** Generating vanilla HybridTopologyFactory ***
INFO:relative:Beginning nonbonded method, total particle, barostat, and exceptions retrieval...
INFO:relative:Flattening torsions of unique new/old at lambda = 0/1


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.4083978161843684), ('CustomAngleForce', 1.622079400619289), ('CustomTorsionForce', 17.50313784099423), ('CustomBondForce', -81.92370790016638)]


INFO:relative:Flattening exceptions of unique new/old at lambda = 0/1
INFO:relative:Old system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:New system forces: dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce', 'MonteCarloBarostat'])
INFO:relative:No unknown forces.
INFO:relative:Nonbonded method to be used (i.e. from old system): 4
INFO:relative:Adding and mapping old atoms to hybrid system...
INFO:relative:Adding and mapping new atoms to hybrid system...
INFO:relative:Added MonteCarloBarostat.
INFO:relative:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]
INFO:relative:Determined atom classes.
INFO:relative:Generating old system exceptions dict...
INFO:relative:Gene

In [25]:
complex_htf = solvent_delivery.get_complex_rhtf_0()
pickle.dump(complex_htf, open("T42A_complex_0.pickle", "wb" ))

In [26]:
# Create REST-ified hybrid system
rest_atoms = list(complex_htf._atom_classes['unique_old_atoms']) + list(complex_htf._atom_classes['unique_new_atoms'])
factory = RESTTopologyFactoryV3(complex_htf.hybrid_system, complex_htf.hybrid_topology, rest_region=rest_atoms, use_dispersion_correction=True)
REST_system = factory.REST_system

# Compare energy components
compare_energy_components(REST_system, complex_htf.hybrid_system, complex_htf.hybrid_positions)


INFO:REST:*** Generating RESTTopologyFactoryV3 ***
INFO:REST:No unknown forces.
INFO:REST:alpha_ewald is 2.7569734238004693
INFO:REST:Added MonteCarloBarostat.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=6.7997, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.7997, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.7997), unit=nanometer)]


conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
[('HarmonicBondForce', 189.90619254301455), ('HarmonicAngleForce', 974.9962574395616), ('PeriodicTorsionForce', 3830.525263094371), ('NonbondedForce', 1009477.8320700994), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
[('CustomBondForce', 189.90619254301455), ('CustomAngleForce', 974.9962574395629), ('CustomTorsionForce', 3830.5252630943774), ('CustomNonbondedForce', -144874.33555218598), ('CustomNonbondedForce', 1164920.788917851), ('CustomBondForce', 809622.4491905153), ('NonbondedForce', -820190.5439900638), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
1009477.8320700994
1009478.3585661164
Energy bookkeeping was a success!


## Double check that groups = 1 << i is the same as groups = set{[i]}

In [25]:
from openmmtools import utils
DEFAULT_PLATFORM = utils.get_fastest_platform()
def compute_potential_components2(context, beta = beta, platform = DEFAULT_PLATFORM):
    """
    Compute potential energy, raising an exception if it is not finite.
    Parameters
    ----------
    context : simtk.openmm.Context
        The context from which to extract, System, parameters, and positions.
    """
    # Make a deep copy of the system.
    import copy

    from perses.dispersed.utils import configure_platform
    platform = configure_platform(platform.getName(), fallback_platform_name='Reference', precision='double')

    system = context.getSystem()
    system = copy.deepcopy(system)
    # Get positions.
    positions = context.getState(getPositions=True).getPositions(asNumpy=True)
    # Get Parameters
    parameters = context.getParameters()
    # Segregate forces.
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        force.setForceGroup(index)
    # Create new Context.
    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    context = openmm.Context(system, integrator, platform)
    context.setPositions(positions)
    for (parameter, value) in parameters.items():
        context.setParameter(parameter, value)
    energy_components = list()
    for index in range(system.getNumForces()):
        force = system.getForce(index)
        forcename = force.__class__.__name__
        groups = 1<<index
        potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
        energy_components.append((forcename, potential))
    groups = 1<<31
    potential = beta * context.getState(getEnergy=True, groups=groups).getPotentialEnergy()
    energy_components.append(('reciprocal', potential))
    del context, integrator
    return energy_components

In [22]:
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT

In [23]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(ala.positions)

compute_potential_components2(context_other)

conducting subsequent work with the following platform: CPU


[('HarmonicBondForce', 0.03454256736202116),
 ('HarmonicAngleForce', 0.6071512895607732),
 ('PeriodicTorsionForce', 3.2298499815095068),
 ('NonbondedForce', -9889.983001429402),
 ('AndersenThermostat', 0.0),
 ('reciprocal', 0.0)]

In [26]:
## CASE 2: alanine dipeptide in solvent
# Create vanilla system
ala = AlanineDipeptideExplicit()
system = ala.system
system.removeForce(4)
# system.getForce(3).setNonbondedMethod(0)
positions = ala.positions
topology = ala.topology

system.getForce(3).setReciprocalSpaceForceGroup(31) # Move reciprocal space to group 31

## COMMENT IN THE 2 LINES IN COMPUTE_POTENTIAL_COMPONENTS2 ##

# Create thermodynamic state and sampler state for non-rest system
thermostate_other = ThermodynamicState(system=system, temperature=temperature)

# Create context for non-rest system
integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
context_other = thermostate_other.create_context(integrator_other)
context_other.setPositions(ala.positions)

compute_potential_components2(context_other)

conducting subsequent work with the following platform: CPU


[('HarmonicBondForce', 0.03454256736202116),
 ('HarmonicAngleForce', 0.6071512895607732),
 ('PeriodicTorsionForce', 3.2298499815095068),
 ('NonbondedForce', 70688.90357261203),
 ('AndersenThermostat', 0.0),
 ('reciprocal', -80578.88657404146)]

The reciprocal and nonbonded force energies here, when added together, match the og nonbonded force energy. And non of the valence energies have changed. Therefore, groups=1 << i is the same as groups = set{[i]} and the way im setting the reciprocal space group is correct.